In [15]:
from numpy import char
from datetime import date
import pandas as pd
import numpy as np
pd.set_option('display.float_format', str)
import time

In [16]:
""" 
from tpch-pgsql-master/query_root/prep_query:
    create_tbl.sql

CREATE TABLE LINEITEM (
    L_ORDERKEY        INTEGER NOT NULL, -- references O_ORDERKEY
    L_PARTKEY        INTEGER NOT NULL, -- references P_PARTKEY (compound fk to PARTSUPP)
    L_SUPPKEY        INTEGER NOT NULL, -- references S_SUPPKEY (compound fk to PARTSUPP)
    L_LINENUMBER    INTEGER,
    L_QUANTITY        DECIMAL,
    L_EXTENDEDPRICE    DECIMAL,
    L_DISCOUNT        DECIMAL,
    L_TAX            DECIMAL,
    L_RETURNFLAG    CHAR(1),
    L_LINESTATUS    CHAR(1),
    L_SHIPDATE        DATE,
    L_COMMITDATE    DATE,
    L_RECEIPTDATE    DATE,
    L_SHIPINSTRUCT    CHAR(25),
    L_SHIPMODE        CHAR(10),
    L_COMMENT        VARCHAR(44)
);

and
    create_idx.sql
    
ALTER TABLE LINEITEM ADD PRIMARY KEY (L_ORDERKEY, L_LINENUMBER);
"""

columnnames = ["L_ORDERKEY", "L_PARTKEY", "L_SUPPKEY", "L_LINENUMBER", "L_QUANTITY", "L_EXTENDEDPRICE", "L_DISCOUNT", "L_TAX",
               "L_RETURNFLAG", "L_LINESTATUS", "L_SHIPDATE", "L_COMMITDATE", "L_RECEIPTDATE", "L_SHIPINSTRUCT", "L_SHIPMODE", "L_COMMENT"]

for i in range(len(columnnames)):
    columnnames[i] = columnnames[i].lower()

data_types = {
    'l_orderkey': int,
    'l_partkey': int,
    'l_suppkey': int,
    'l_linenumber': int,
    'l_quantity': float,
    'l_extendedprice': float,
    'l_discount': float,
    'l_tax': float,
    'l_returnflag': str,
    'l_linestatus': str,
    'l_shipinstruct': str,
    'l_shipmode': str,
    'l_comment': str
}


In [17]:
# Don't set indexes, as we can't access them with Pandas selection!
lineitem = pd.read_table("../data_storage/lineitem.tbl.csv", sep="|",
                         names=columnnames, dtype=data_types, parse_dates=['l_shipdate', 'l_commitdate', 'l_receiptdate'])


## Q6


In [18]:
# Hyper Pandas
start_time = time.time()

df_scan_1 = lineitem[(lineitem.l_quantity < 24.0) & ((lineitem.l_discount.between(0.05, 0.07, inclusive='both')) & (lineitem.l_shipdate.between('1994-01-01', '1995-01-01', inclusive='left')))]
df_scan_1 = df_scan_1[['l_orderkey', 'l_partkey', 'l_suppkey', 'l_linenumber', 'l_quantity', 'l_extendedprice', 'l_discount', 'l_tax', 'l_returnflag', 'l_linestatus', 'l_shipdate', 'l_commitdate', 'l_receiptdate', 'l_shipinstruct', 'l_shipmode', 'l_comment']]
df_scan_1['l_extendedpricel_discount'] = df_scan_1.l_extendedprice * df_scan_1.l_discount
df_group_1 = pd.DataFrame()
df_group_1['l_extendedpricel_discountsum'] = [df_scan_1.l_extendedpricel_discount.sum()]
df_output_1 = df_group_1
df_output_1 = df_output_1.rename(columns={'l_extendedpricel_discountsum': 'revenue'})
df_output_1 = df_output_1[['revenue']]

end_time = time.time()
print(df_output_1.head(1))

print("--- %s seconds ---" % (end_time - start_time))
# Should be: 
#              revenue
# 0 123141078.22829999
#

             revenue
0 123141078.22829999
--- 0.22750139236450195 seconds ---
